**Prediction task**

We are interested in predicting the future income from a user.
1. Please create a prediction model, aiming to predict the target variable (org_price_usd_following_30_days). Use the train set for training a model, aiming to minimize RMSE of predictions over the test set.
2. What are the three most important features that contributed to the prediction?

Note: the following columns are related to the next task, and should not be used in the current task: ”treatment”, “org price usd following 30 days after impact”.

In [1]:
!pip install pandas sweetviz scikit-learn xgboost catboost matplotlib lightgbm numpy

For LightGBM, you may need to install libomp via Homebrew:
1. Run `brew install libomp` in your terminal.
2. Export the variables suggested by brew to your shell configuration (e.g., .zshrc or .bashrc):
    echo 'export LDFLAGS="-L/usr/local/opt/libomp/lib"' >> ~/.zshrc
    echo 'export CPPFLAGS="-I/usr/local/opt/libomp/include"' >> ~/.zshrc
    source ~/.zshrc
 3. Then, install LightGBM with: `pip install lightgbm --no-binary lightgbm`

In [2]:
import pandas as pd
import sweetviz as sv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
print("LightGBM successfully imported!")
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

/Users/michaelbenmergui/Documents/Gioia/PycharmProjects/data-wizard-gioia/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LightGBM successfully imported!


In [3]:
df_train = pd.read_csv('train_home_assignment_.csv', index_col=0)
df_test = pd.read_csv('test_home_assignment.csv')

In [4]:
# drop columns related to next task
df_train.drop(columns=['treatment', 'org_price_usd_following_30_days_after_impact'], inplace=True)

I'll be carrying EDA, Feature treatment (preprocessing, engineering and selection based on feature importance) on the train set. I assume train and test set have the same prediction point, meaning the explanatory features available in the train set will match the prediction point in the user funnel within the gaming app where my model will have to return the prediction for new data (test set). For example, if the user enters the gaming app, plays and we want to return a prediction before the next level unlocks, I assume that the features available in this dataset are all available before the next level unlocks. In any case the dimensions of train and test the datasets match (with the same columns), so I assume the features used in the train set match the prediction point of the test set.

I'll further split the train set into train and validation, to assess first of all the performance of the model on the validation set and use the latter for tuning eventual hyperparameters. I'll leave the test set untouched, and I'll address it as new data, meaning the test set will go through relevant transformations or preprocessing based on the train set before inputting it into the tuned model for final predictions.

Finally, I'll evaluate the tuned model with the chosen features on the test set by comparing the RMSE and other relevant metrics between different learners.

In [5]:
#check if we have the same columns in train and test
# Get the column sets
train_columns = set(df_train.columns)
test_columns = set(df_test.columns)

# Check for equality
if train_columns == test_columns:
    print("The columns in df_train and df_test are the same.")
else:
    print("The columns in df_train and df_test are different.")

    # Find columns in df_train but not in df_test
    missing_in_test = train_columns - test_columns
    if missing_in_test:
        print(f"Columns in df_train but not in df_test: {missing_in_test}")

    # Find columns in df_test but not in df_train
    missing_in_train = test_columns - train_columns
    if missing_in_train:
        print(f"Columns in df_test but not in df_train: {missing_in_train}")

The columns in df_train and df_test are the same.


EDA

In this section I use sweetviz, a module that returns a html with a comprehensive exploratory data analysis including:
- marginal and joint distributions of Y, continuous features and categorical features
- measures of associations
- the goal here is to:
    - check eventual missing values (for imputation)
    - check statistical outliers for capping or removal
    - look at the correlations:
        - with Y, to have a first glimpse of the predictive power of the feature (we ideally want features in X being highly correlated with Y in its absolute value).
        - between features, to get an idea if we need to introduce interaction variables for highly correlated explanatory variables (ideally we want explanatory features independent between each other, so we do not want multicollinearity)
    - check if we have constant features, which having 0 variance are not explanatory at all
    - the following checks will help us chose whether we are in a linear or non-linear setting, and therefore the choice of the learners
        - check the skewness of the label to assess if a transformation is needed
        - check the scale of the features and the label.
        - check the relationship between Y and the features
    - check cardinality of categorical features for binning or one-hot-encoding
    - check if we have duplicate rows

In [6]:
# Specify data types. everything that is count, total, occurrence, price, number of days - I address it as continuous
for col in df_train.columns:
    if col in ['weekday', 'village_id']:
        df_train[col] = df_train[col].astype('category')
    else:
        df_train[col] = df_train[col].astype(float)


report = sv.analyze([df_train, "Train"], target_feat="org_price_usd_following_30_days")
report.show_html("regression_report.html")

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:07 -> (00:00 left)                       


Report regression_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [7]:
# further check correlations
correlation_matrix = df_train.corr()
print(correlation_matrix['org_price_usd_following_30_days'].sort_values(ascending=False))

org_price_usd_following_30_days                                1.000000
org_price_usd_preceding_30_days                                0.728005
spins_reward_preceding_30_days                                 0.716380
pet_xp_reward_preceding_30_days                                0.689277
org_price_usd_preceding_7_to_30_days                           0.675280
org_price_usd_triple_preceding_30_days                         0.633510
payment_occurrences_preceding_30_days                          0.619148
tournament_spins_reward_7_preceding                            0.612171
org_price_usd_preceding_3_days                                 0.583488
org_price_usd_preceding_3_to_7_days                            0.577685
payment_occurrences_preceding_7_to_30_days                     0.568057
chests_reward_preceding_30_days                                0.508032
payment_occurrences_preceding_3_days                           0.458481
payment_occurrences_preceding_3_to_7_days                      0

We have some features that are highly correlated with Y, but that are highly correlated with each other. we address them later on. Fo example:

In [8]:
df_train[['org_price_usd_preceding_30_days','spins_reward_preceding_30_days']].corr()

,org_price_usd_preceding_30_days,spins_reward_preceding_30_days
org_price_usd_preceding_30_days,1.000000,0.973278
spins_reward_preceding_30_days,0.973278,1.000000


Observations from EDA (in case the html does not work, I added the pictures_EDA folder with relevant screenshots):

1. The Label, in black: first of all, it is very skewed (43.3% are zeros). And this is reflected by the skewness of 22.5. With a skewness this high, a transformation like log or sqrt will decrease the skewness but won't make the distribution symmetrical, which means that maybe a linear setting might not be ideal.
2. For each feature, the report gives the following indexes to measure correlations:

    a. Theil's U uncertainty coefficient: between categorical variables

        - values are very low - which means that the features are independent.

    b. Correlation Ratio (chi squared): between categorical and numerical variables

        - the categorical variables (village_id, weekday) show a mild-weak association with Y.

    c. Pearson Correlation: between numerical variables

        - we can see that there are 2 features which are highly correlated with the label: org_price_usd_preceding_30_days (0.73), spins_reward_preceding_30_days (0.72).
        - below them, 9 more features with moderate correlation (between 0.5 and 0.7)
        - we do have multicolinearity

3. A short explanation about the plots: each feature is depicted with both a histogram (feature values on the x axis, frequency (%) of feature value on the left vertical axis) and a line plot with aggregated data (feature values on the x axis, average Y value on the right vertical axis). We can see that many of the features present a non-linear relationship with Y.


ACTION ITEMS
- drop duplicates rows
- drop constant feature: spins_rewards_lo_preceding_7_days
- There are some features that sweetviz addressed as categorical: ['active_days_preceding_7_days', 'active_days_preceding_7_to_14_days', 'total_set_completed', 'total_friend_link_invites']. I am not sure they are, so I computed for them the pearson correlation which still resulted very low. I address these features as numerical.
- village_id and weekday: I'll bin them according to sweetviz suggestion into less categories considering the high cardinality and I apply OHE, leaving out the "other" category for redundancy.
- we don't have missing values. so no imputation needed.
- we have one feature with negative values: ltv_gross_up_to_preceding_30_days
- address highly correlated features: features that are highly correlated with Y are used to generate interaction features. features that have lower correlations with Y are dropped.
- scaling (for skewness): I'll apply log transformation and minmax scaling to numerical features
- outliers: I'll cap outliers according to IQR
- variance: I'll drop features whose variance is below a given threshold (0.01 and 0.1 for binary and numerical features accordingly).

Considering the setting is non-linear, that features and label have different scales, that we might have outliers given the presence of high average Y values in the line plots, and considering the presence of highly correlated features, it seems like tree-based learners would be a better fit for this problem. Tree based learners are robust to outliers and correlated features, and are scale indifferent - but I'm going to address this anyway.

In [9]:
# drop constant features
df_train.drop(columns=['spins_rewards_lo_preceding_7_days'], inplace=True)

In [10]:
# drop duplicates
df_train.drop_duplicates(inplace=True)

In [11]:
#split into train-dev set
X = df_train.drop(columns=['org_price_usd_following_30_days'])
y = df_train['org_price_usd_following_30_days']

In [12]:
def preprocess_binning_and_encoding(df, target_column, top_n=None, categories_to_keep=None, fit=True, prefix=None):
    """
    Generalized function for binning and encoding a categorical column.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to preprocess.
    - target_column (str): The column to bin and encode.
    - top_n (int): Number of top categories to keep based on frequency (fit phase uses this to determine categories_to_keep).
                   Ignored if categories_to_keep is provided.
    - categories_to_keep (list): List of categories to keep (if provided, fit mode skips determining categories).
    - fit (bool): Whether to determine categories_to_keep (True for training dataset, False for dev/test dataset).
    - prefix (str): Prefix for created columns during one-hot encoding. Defaults to the target_column name.

    Returns:
    - pd.DataFrame: The processed DataFrame with the binned and encoded column.
    - categories_to_keep (list): The list of categories kept during the fit process (if fit=True).
    """
    df = df.copy()  # Work on a copy to prevent changes to the original DataFrame

    # Step 1: Convert the target column to string if not already
    df[target_column] = df[target_column].astype(int).astype(str)

    # Step 2: Determine the categories to keep (fit phase)
    if fit:
        if categories_to_keep is None:
            # Automatically determine top categories by frequency
            categories_to_keep = df[target_column].value_counts().index.tolist()[:top_n]

    # Step 3: Bin the target column
    binned_column = f"{target_column}_binned"
    df[binned_column] = df[target_column].apply(
        lambda x: x if x in categories_to_keep else 'Other'
    )

    # Step 4: One-hot encode the binned column
    prefix = prefix if prefix else target_column
    df_encoded = pd.get_dummies(df, columns=[binned_column], prefix=prefix)

    # Step 5: Ensure consistent columns across datasets (fixed based on categories_to_keep)
    fixed_columns = [f"{prefix}_{cat}" for cat in categories_to_keep]
    for col in fixed_columns:
        if col not in df_encoded:
            df_encoded[col] = 0  # Add missing columns with default value 0

    # Remove unnecessary columns (like 'Other')
    extra_columns = [col for col in df_encoded.columns if col.startswith(f"{prefix}_") and col not in fixed_columns]
    df_encoded = df_encoded.drop(columns=extra_columns)

    # Step 6: Drop the original target column
    df_encoded = df_encoded.drop(columns=[target_column, binned_column], errors="ignore")

    # Step 7: Reorder the one-hot encoded columns
    df_encoded = df_encoded[fixed_columns]

    # Step 8: Merge back with the original DataFrame (other columns untouched)
    df = pd.concat([df.drop(columns=[target_column, binned_column], errors="ignore"), df_encoded], axis=1)

    # Return the processed DataFrame and, if in fit mode, the categories_to_keep
    if fit:
        return df, categories_to_keep
    else:
        return df

In [13]:
class LogTransformer:
    def __init__(self):
        # List of features to transform
        self.features = None
        # Dictionary to store shift values for each feature
        self.shifts = {}

    def fit(self, df, features):
        """
        Check and store the features that will be log-transformed.

        Parameters:
        - df (pd.DataFrame): The training dataset
        - features (list): Non-binary numerical features for log transformation

        Returns:
        - None
        """
        self.features = features
        # Calculate and store the shift value for each feature (if needed)
        for feature in features:
            min_value = df[feature].min()
            self.shifts[feature] = abs(min_value) + 1 if min_value < 0 else 0

    def transform(self, df):
        """
        Apply the log transformation to the stored features.

        Parameters:
        - df (pd.DataFrame): The dataset to transform

        Returns:
        - pd.DataFrame: Dataset with log-transformed features
        """
        df = df.copy()  # Avoid modifying the original DataFrame
        if self.features is None:
            raise ValueError("You need to fit the transformer before applying transform.")
        # Apply log transformation with shifting (if necessary)
        for feature in self.features:
            shift_value = self.shifts.get(feature, 0)
            df[feature] = np.log1p(df[feature] + shift_value)
        return df

In [14]:
# Outlier capping with IQR - Generalized for Fit/Transform
class OutlierCapper:
    def __init__(self):
        # Dictionary to store the lower and upper bounds for each feature
        self.bounds_ = {}

    def fit(self, df, features):
        """
        Compute the IQR-based outlier bounds for the given features.

        Parameters:
        - df (pd.DataFrame): The training dataset
        - features (list): List of feature names to compute bounds for

        Returns:
        - None: Saves bounds to self.bounds_
        """
        for feature in features:
            # Compute Q1 (25th percentile) and Q3 (75th percentile)
            Q1 = df[feature].quantile(0.25)
            Q3 = df[feature].quantile(0.75)
            # Compute Interquartile Range (IQR)
            IQR = Q3 - Q1
            # Define lower and upper bounds for outlier capping
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            # Store the bounds for the feature
            self.bounds_[feature] = (lower_bound, upper_bound)

    def transform(self, df, features):
        """
        Apply the precomputed bounds to cap outliers in the DataFrame.

        Parameters:
        - df (pd.DataFrame): The dataset to apply the bounds to
        - features (list): List of feature names to apply bounds to

        Returns:
        - pd.DataFrame: Dataset with capped values
        """
        df = df.copy()  # Avoid modifying the original dataset
        for feature in features:
            if feature in self.bounds_:
                lower_bound, upper_bound = self.bounds_[feature]
                # Cap the outliers to the precomputed bounds
                df[feature] = np.where(df[feature] < lower_bound, lower_bound, df[feature])
                df[feature] = np.where(df[feature] > upper_bound, upper_bound, df[feature])
        return df

Further split train into train-dev

In [15]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# For village_id, keep the top 13 categories
X_train_processed, village_ids_to_keep = preprocess_binning_and_encoding(
    df=X_train,
    target_column="village_id",
    top_n=13,
    fit=True
)
# For weekday, keep top 4 categories
X_train_processed, weekdays_to_keep = preprocess_binning_and_encoding(
    df=X_train_processed,
    target_column="weekday",
    top_n=4,
    fit=True
)

In [17]:
# Preprocess dev set for village_id
X_dev_processed = preprocess_binning_and_encoding(
    df=X_dev,
    target_column="village_id",
    categories_to_keep=village_ids_to_keep,
    fit=False
)

# Preprocess dev set for weekday
X_dev_processed = preprocess_binning_and_encoding(
    df=X_dev_processed,
    target_column="weekday",
    categories_to_keep=weekdays_to_keep,
    fit=False
)

In [18]:
# Recompute numerical features from the processed DataFrame
numerical_features = X_train_processed.select_dtypes(include=np.number).columns.tolist()

# Binary features from one-hot encoding (they shouldn't be capped)
binary_features = [
    col for col in X_train_processed.columns
    if col.startswith('weekday_') or col.startswith('village_id_')
]

In [19]:
# Initialize LogTransformer, addressing columns with negative values
log_transformer = LogTransformer()

# Fit log transformer on X_train_cleaned
log_transformer.fit(X_train_processed, numerical_features)

# Apply log transformation to train, dev, and test datasets
X_train_log_transformed = log_transformer.transform(X_train_processed)
X_dev_log_transformed = log_transformer.transform(X_dev_processed)

In [20]:
# Initialize OutlierCapper
outlier_capper = OutlierCapper()

# Fit the capper on the training set
outlier_capper.fit(X_train_log_transformed, numerical_features)

# Transform train, dev, and test sets
X_train_cleaned = outlier_capper.transform(X_train_log_transformed, numerical_features)
X_dev_cleaned = outlier_capper.transform(X_dev_log_transformed, numerical_features)

In [21]:
# Step 1: Initialize separate VarianceThreshold selectors
binary_selector = VarianceThreshold(threshold=0.01)  # For binary features
numerical_selector = VarianceThreshold(threshold=0.1)  # For numerical features

# Step 2: Fit selectors on the training data
binary_selector.fit(X_train_log_transformed[binary_features])  # Fit on binary features
numerical_selector.fit(X_train_log_transformed[numerical_features])  # Fit on numerical features

final_selected_features = (
        list(pd.Index(binary_features)[binary_selector.get_support()]) +  # Selected binary features
        list(pd.Index(numerical_features)[numerical_selector.get_support()])  # Selected numerical features
)

if len(final_selected_features) == 0:
    raise ValueError("No features were selected. Consider lowering the variance thresholds.")

# Step 4: Transform train, dev, and test sets
X_train_selected = X_train_log_transformed[final_selected_features]  # Only selected features from train set
X_dev_selected = X_dev_log_transformed[final_selected_features]  # Subset dev set based on `final_selected_features`

# Print selected features
print(f"Selected Features ({len(final_selected_features)} in total):")
print(final_selected_features)

Selected Features (61 in total):
['village_id_0', 'village_id_170', 'village_id_165', 'village_id_168', 'village_id_166', 'village_id_160', 'village_id_169', 'village_id_175', 'village_id_167', 'village_id_172', 'village_id_161', 'village_id_171', 'village_id_162', 'weekday_4', 'weekday_3', 'weekday_5', 'weekday_2', 'payment_occurrences_preceding_30_days', 'hours_since_installed_ma', 'total_raids_preceding_7_days', 'org_price_usd_preceding_3_days', 'hours_in_village', 'total_card_xp', 'total_villages_completed_preceding_7_days', 'total_friends_active_in_the_last_7_to_14_days', 'chests_reward_preceding_30_days', 'tournament_number_of_rank_in_top_10_7_preceding', 'pet_xp_reward_preceding_30_days', 'total_set_completed_preceding_7_days', 'total_oos_action_fire_preceding_7_days', 'total_spins_preceding_7_days', 'total_card_xp_diff_preceding_7_days', 'tournament_spins_reward_7_preceding', 'avg_spin_aggressiveness_oos_preceding_7_days', 'total_pets_feed_preceding_7_days', 'active_days_preced

In [22]:
# Identify unselected features for binary features
binary_unselected_features = [
    column for i, column in enumerate(binary_features)
    if not binary_selector.get_support()[i]
]

# Identify unselected features for numerical features
numerical_unselected_features = [
    column for i, column in enumerate(numerical_features)
    if not numerical_selector.get_support()[i]
]

# Combine unselected features
unselected_features = binary_unselected_features + numerical_unselected_features

# Print the unselected features along with their count
print(f"Unselected Features ({len(unselected_features)} in total):")
print(unselected_features)

Unselected Features (6 in total):
['total_villages_completed', 'ltv_gross_up_to_preceding_30_days', 'total_friend_link_invites', 'tournament_coins_reward_7_preceding', 'total_set_completed', 'total_oos_action_fire']


In [23]:
# handle multicollinearity by generating interaction terms, and dropping features less correlated to Y

def generate_interaction_terms(df):

    df['interaction_org_price_usd_spin_rewards_preceding_30_days'] = df['org_price_usd_preceding_30_days'] * df['spins_reward_preceding_30_days'] * df['payment_occurrences_preceding_30_days']

    df['interaction_preceding_3_days'] = df['payment_occurrences_preceding_3_days'] * df['org_price_usd_preceding_3_days']

    df.drop(columns=['org_price_usd_preceding_30_days', 'spins_reward_preceding_30_days', 'payment_occurrences_preceding_30_days', 'payment_occurrences_preceding_3_days', 'org_price_usd_preceding_3_days', 'org_price_usd_preceding_7_to_30_days', 'payment_occurrences_preceding_7_to_30_days'], inplace=True)

    return df

In [24]:
X_train_selected = generate_interaction_terms(df=X_train_selected)
X_dev_selected = generate_interaction_terms(df=X_dev_selected)

/var/folders/tz/4zg0f6x90m7cbq4q1p10p1m40000gn/T/ipykernel_36209/2653920461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['interaction_org_price_usd_spin_rewards_preceding_30_days'] = df['org_price_usd_preceding_30_days'] * df['spins_reward_preceding_30_days'] * df['payment_occurrences_preceding_30_days']
/var/folders/tz/4zg0f6x90m7cbq4q1p10p1m40000gn/T/ipykernel_36209/2653920461.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['interaction_preceding_3_days'] = df['payment_occurrences_precedi

In [25]:
# Step 1: Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))  # Adjust range if needed

# Step 2: Separate numerical features (non-binary) for scaling
numerical_to_scale = [feature for feature in numerical_features if feature in X_train_selected.columns]

# Step 3: Fit the scaler on the numerical features of the training set
X_train_selected_numerical_scaled = X_train_selected.copy()  # Copy to avoid modifying the original data
X_train_selected_numerical_scaled[numerical_to_scale] = scaler.fit_transform(X_train_selected[numerical_to_scale])

# Step 4: Apply the scaler to dev and test sets
X_dev_selected_scaled = X_dev_selected.copy()
X_dev_selected_scaled[numerical_to_scale] = scaler.transform(X_dev_selected[numerical_to_scale])

In [26]:
X_train = X_train_selected_numerical_scaled.copy()
X_dev = X_dev_selected_scaled.copy()

Tree based feature importance

In [27]:
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, y_train)

importances = gb.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
cumulative_importance = np.cumsum(importances[sorted_indices])
threshold_index = np.where(cumulative_importance >= 0.95)[0][0]
selected_features = X_train.columns[sorted_indices[:threshold_index + 1]]

In [28]:
len(selected_features)

12

In [29]:
print(f"Selected Features (95% cumulative importance): {list(selected_features)}")

Selected Features (95% cumulative importance): ['interaction_org_price_usd_spin_rewards_preceding_30_days', 'tournament_spins_reward_7_preceding', 'org_price_usd_triple_preceding_30_days', 'interaction_preceding_3_days', 'pet_xp_reward_preceding_30_days', 'org_price_usd_preceding_3_to_7_days', 'hours_since_installed_ma', 'chests_reward_preceding_30_days', 'total_spins_preceding_7_days', 'total_villages_completed_preceding_7_days', 'total_card_xp', 'avg_past_seen_price_oos_preceding_7_days']


In [30]:
X_train[selected_features].corr()

,interaction_org_price_usd_spin_rewards_preceding_30_days,tournament_spins_reward_7_preceding,org_price_usd_triple_preceding_30_days,interaction_preceding_3_days,pet_xp_reward_preceding_30_days,org_price_usd_preceding_3_to_7_days,hours_since_installed_ma,chests_reward_preceding_30_days,total_spins_preceding_7_days,total_villages_completed_preceding_7_days,total_card_xp,avg_past_seen_price_oos_preceding_7_days
interaction_org_price_usd_spin_rewards_preceding_30_days,1.000000,0.391211,0.757776,0.580668,0.518192,0.605447,0.058506,0.810140,0.187956,0.061099,0.083262,0.296680
tournament_spins_reward_7_preceding,0.391211,1.000000,0.344838,0.411001,0.280371,0.514908,0.073770,0.300493,0.368017,0.238756,0.092398,0.311000
org_price_usd_triple_preceding_30_days,0.757776,0.344838,1.000000,0.395057,0.686086,0.471256,0.266369,0.538212,0.126432,-0.121117,0.062466,0.289122
interaction_preceding_3_days,0.580668,0.411001,0.395057,1.000000,0.265688,0.367012,-0.025045,0.447752,0.221539,0.236906,0.084992,0.218036
pet_xp_reward_preceding_30_days,0.518192,0.280371,0.686086,0.265688,1.000000,0.325781,0.229785,0.351481,0.129724,-0.064288,0.057428,0.222487
org_price_usd_preceding_3_to_7_days,0.605447,0.514908,0.471256,0.367012,0.325781,1.000000,0.022655,0.452369,0.260433,0.224170,0.060157,0.285095
hours_since_installed_ma,0.058506,0.073770,0.266369,-0.025045,0.229785,0.022655,1.000000,0.035472,-0.120826,-0.504087,-0.038457,0.047079
chests_reward_preceding_30_days,0.810140,0.300493,0.538212,0.447752,0.351481,0.452369,0.035472,1.000000,0.136834,0.026103,0.068902,0.201500
total_spins_preceding_7_days,0.187956,0.368017,0.126432,0.221539,0.129724,0.260433,-0.120826,0.136834,1.000000,0.500875,0.208040,0.411977
total_villages_completed_preceding_7_days,0.061099,0.238756,-0.121117,0.236906,-0.064288,0.224170,-0.504087,0.026103,0.500875,1.000000,0.109440,0.183388


In [31]:
X_train_selected = X_train[selected_features]
X_dev_selected = X_dev[selected_features]

Reasons why I used Gradient Boosting for feature selection:

Random Forest is less suitable for feature selection because it builds trees independently, which means that if we have multiple correlated features, they can be selected in different trees and inflate the importance. GB instead works sequentially, this means that if we have 2 features that are highly correlated, GB selects one to avoid redundancy. The feature selected is the one that reduces the residual error at each step. Then once one feature in a correlated group is selected, the others offer diminishing returns in terms of residual reduction.

In [32]:
# prepare assessment metrics
def evaluate_metrics(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    return {"RMSE": rmse}

Here I'll use RF as baseline, and XGBoost, lightGBM and Catboost as learners. I do not use GB as learner to avoid bias.

In [ ]:
# Algorithms to evaluate (adaboost does not handle well multicolinearity)
models = {
    "RandomForest": RandomForestRegressor(random_state=42), # baseline
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(verbose=0, random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42),
}

# Evaluate each model on dev set
results = {}
for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_dev_selected)
    metrics = evaluate_metrics(y_dev, y_pred)
    results[name] = metrics

# Print results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()

In [ ]:
# Evaluate each model on train set to check overfitting
results = {}
for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_train_selected)
    metrics = evaluate_metrics(y_train, y_pred)
    results[name] = metrics

# Print results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()

So far it seems like lightGBM is the one that performs better, because of the lowest discrepancy between train and dev RMSE.

Hyperparameter tuning on dev set through RandomSearchCV (for speed, as GridSearchCV takes time) - done on lightGBM.

In [ ]:
lgbm_params = {
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [50, 100, 200],
    "num_leaves": [20, 31, 50],
    "max_depth": [3, 5, 7],
    "min_data_in_leaf": [10, 20, 30]
}

- learning_rate: Smaller values reduce overfitting but require more trees.
- n_estimators: Number of boosting rounds.
- max_depth: Maximum depth of trees.
- num_leaves: Number of leaves in a tree. Lower values reduce overfitting.
- min_data_in_leaf: Minimum samples in a leaf.

In [ ]:
light_gbm = LGBMRegressor(verbose=0, random_state=42)
light_gbm_random_search = RandomizedSearchCV(
    estimator=light_gbm,
    param_distributions=lgbm_params,
    n_iter=50,  # Number of random combinations to try
    cv=5,
    scoring="neg_root_mean_squared_error",
    verbose=0,
    n_jobs=-1,
    random_state=42
)

light_gbm_random_search.fit(X_dev_selected, y_dev)
print("Best Parameters for LightGBM:", light_gbm_random_search.best_params_)
print("Best CV RMSE:", -light_gbm_random_search.best_score_)

Predict on the test set:

In [ ]:
X_test = df_test.drop(columns=['org_price_usd_following_30_days'])
y_test = df_test['org_price_usd_following_30_days']

# Preprocess test data (using same logic)
X_test_processed = preprocess_binning_and_encoding(
    df=X_test,
    target_column="village_id",
    categories_to_keep=village_ids_to_keep,
    fit=False
)

X_test_processed = preprocess_binning_and_encoding(
    df=X_test_processed,
    target_column="weekday",
    categories_to_keep=weekdays_to_keep,
    fit=False
)

X_test_log_transformed = log_transformer.transform(X_test_processed)
X_test_cleaned = outlier_capper.transform(X_test_log_transformed, numerical_features)
X_test_selected = X_test_cleaned[final_selected_features]
X_test_selected = generate_interaction_terms(df=X_test_selected)
X_test_selected_scaled = X_test_selected.copy()
X_test_selected_scaled[numerical_to_scale] = scaler.transform(X_test_selected_scaled[numerical_to_scale])
X_test_selected = X_test_selected_scaled[selected_features]

In [ ]:
model = LGBMRegressor(num_leaves=31, n_estimators=200, min_data_in_leaf=10, max_depth=5, learning_rate=0.1, random_state=42, verbose=0)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)
metrics = evaluate_metrics(y_test, y_pred)
print(metrics)

Since the distribution of y is very skewed, the RMSE needs to be assessed by portion of data.

In [ ]:
# Percentile-based scaling
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
p90 = np.percentile(y_test, 90)
p10 = np.percentile(y_test, 10)
relative_rmse_percentile = rmse / (p90 - p10)
print(f"Relative RMSE (Scaled by 90th-10th Percentile): {relative_rmse_percentile:.4f}")

The metric above confirms that the model performs well on the overall span of the data. The percentile based scaling says that the RMSE is < 1% between the 10th and the 90th percentile, meaning that it performs well on the majority of the target values, excluding outliers.

- What are the three most important features that contributed to the prediction?

In [ ]:
# top 3 most important features
importances = model.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
selected_features = X_test_selected.columns[sorted_indices[:3]]
selected_features

Further steps: if I had more time I would have refactored the code into a .py script, with another script to call with auxiliary functions. The model can be stored in a pickle file locally or in an S3 bucket together with encoders/scalers, for production purposes.

**Recommendation task**

We are interested in increasing the income from users. For that, we ran a randomized experiment where the population was given either a 10 usd offer or 2usd offer (see the "treatment" column), aiming to learn what offer should be given to a user. The experiment yielded a target variable named “org_price_usd_following_30_days_after_impact”, reflecting the result of the experiment in terms of income.
1. For each user in the test data, set the treatment (either 10 or 2) that you believe would maximize the target variable (add a new column for that)
2. What are the three most important features that contributed to the decision to give users a specific treatment?

In [ ]:
df_train = pd.read_csv('train_home_assignment_.csv', index_col=0)
df_test = pd.read_csv('test_home_assignment.csv')

In [ ]:
y_train_recom = y_train.to_frame().join(df_train['org_price_usd_following_30_days_after_impact'], how='left')

In [ ]:
y_train_recom[['org_price_usd_following_30_days','org_price_usd_following_30_days_after_impact']].corr()

In [ ]:
# dropping old label
df_train.drop(columns=['org_price_usd_following_30_days'], inplace=True)
df_test.drop(columns=['org_price_usd_following_30_days'], inplace=True)

From this task I understand that I need to recommend a treatment to each new user of the test set under the constraint of maximizing org_price_usd_following_30_days_after_impact.

In [ ]:
print(df_train.groupby(['treatment'])['org_price_usd_following_30_days_after_impact'].sum())

By looking at the sum of org_price_usd_following_30_days_after_impact for each treatment, it seems like treatment does not have any disciminatory power, meaning including it as a further feature in a predictive model won't help much. Therefore, I'll approach this problem in the following way:
- I'll train 2 models, one for each treatment, where the label is org_price_usd_following_30_days_after_impact.
- I'll output 2 predictions for the test set, one per model
- For each new user in the test set, I'll assign 10 if the prediction for 10 is higher, otherwise 2.
- I'll compute the total predicted outcome based on this method and I'll compare it to a random baseline.

I'll use the previous preprocessing and tuned learner as we have a correlation of 0.99 between previous and current label.

In [ ]:
columns_to_join = df_train[['treatment', 'org_price_usd_following_30_days_after_impact']]

# Left join on the indices of X_train_selected
X_train_selected_recom = X_train_selected.join(columns_to_join, how='left')

In [ ]:
# Split the data into 10 and 2 treatment.
df_train_10 = X_train_selected_recom[X_train_selected_recom.treatment == 10].drop(columns=['treatment'])
df_train_2 = X_train_selected_recom[X_train_selected_recom.treatment == 2].drop(columns=['treatment'])

# Separate data by treatment
X_treatment_10 = df_train_10.drop(columns=['org_price_usd_following_30_days_after_impact'])
y_treatment_10 = df_train_10['org_price_usd_following_30_days_after_impact']

X_treatment_2 = df_train_2.drop(columns=['org_price_usd_following_30_days_after_impact'])
y_treatment_2 = df_train_2['org_price_usd_following_30_days_after_impact']

# Train models for each treatment
model_10 = GradientBoostingRegressor()
model_10.fit(X_treatment_10, y_treatment_10)

model_2 = GradientBoostingRegressor()
model_2.fit(X_treatment_2, y_treatment_2)



In [ ]:
#check if we have the same columns in train and test
# Get the column sets
X_test_selected_recom = X_test_selected.copy()
train_columns = set(X_treatment_10.columns)
test_columns = set(X_test_selected_recom.columns)

# Check for equality
if train_columns == test_columns:
    print("The columns in df_train and df_test are the same.")
else:
    print("The columns in df_train and df_test are different.")

    # Find columns in df_train but not in df_test
    missing_in_test = train_columns - test_columns
    if missing_in_test:
        print(f"Columns in df_train but not in df_test: {missing_in_test}")

    # Find columns in df_test but not in df_train
    missing_in_train = test_columns - train_columns
    if missing_in_train:
        print(f"Columns in df_test but not in df_train: {missing_in_train}")

In [ ]:
# Predict for the test set
y_pred_10 = model_10.predict(X_test_selected_recom)
y_pred_2 = model_2.predict(X_test_selected_recom)

# Assign treatment based on predicted outcomes
X_test_selected_recom["recommended_treatment"] = [10 if pred_10 > pred_2 else 2 for pred_10, pred_2 in zip(y_pred_10, y_pred_2)]

In [ ]:
# Calculate total predicted outcome for recommended treatments
X_test_selected_recom["predicted_outcome"] = [
    pred_10 if treatment == 10 else pred_2
    for treatment, pred_10, pred_2 in zip(X_test_selected_recom["recommended_treatment"], y_pred_10, y_pred_2)
]
total_predicted_outcome = X_test_selected_recom["predicted_outcome"].sum()
print(f"Total Predicted Outcome: {total_predicted_outcome}")

# Calculate uplift (compared to random assignment baseline)
baseline_outcome = max(y_pred_10.mean(), y_pred_2.mean()) * len(X_test_selected_recom)
uplift = total_predicted_outcome - baseline_outcome
print(f"Uplift: {uplift}")

In [ ]:
print(X_test_selected_recom.groupby(["recommended_treatment"])['predicted_outcome'].sum())

What are the three most important features that contributed to the decision to give users a specific treatment?

In [ ]:
# Feature importance for \$10 treatment model
importances_10 = model_10.feature_importances_
features_10 = pd.DataFrame({
    "feature": X_treatment_10.columns,
    "importance": importances_10
}).sort_values(by="importance", ascending=False)

print("Top features for \$10 treatment:")
print(features_10.head(3))

# Feature importance for \$2 treatment model
importances_2 = model_2.feature_importances_
features_2 = pd.DataFrame({
    "feature": X_treatment_2.columns,
    "importance": importances_2
}).sort_values(by="importance", ascending=False)

print("Top features for \$2 treatment:")
print(features_2.head(3))
